Usage of numpy fft to extract features for segmentation. Based on:

HARMONIC ANALYSIS OF TIME-SERIES AVHRR NDVI DATA FOR CHARACTERIZING US GREAT PLAINS LAND USE/LAND COVER

1. Data

Se utiliza la ruta:

`/LUSTRE/MADMEX/tasks/2019_tasks/generacion_datos_entrenamiento_cultivos/examples_source_data`


No se usa la funcionalidad del datacube (gridworkflow load) pues se requiere que estén en la ruta:

`/shared_volume/datacube/datacube_ingest/*/mexico/`

pero el `shared_volume` es de `LUSTRE` y no de la máquina de Ixchel... se prefiere no tener los datos en la ruta de la línea anterior y mejor usar la de `tasks`... al final el load no importa para la creación de las series de tiempo...



In [1]:
from datetime import datetime
import datetime as dt
import xarray as xr
import numpy as np
import glob

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from datacube.api import GridWorkflow


In [2]:
path = '/LUSTRE/MADMEX/tasks/2019_tasks/generacion_datos_entrenamiento_cultivos/examples_source_data/' 

In [3]:
sr_0_no_chunks = xr.open_mfdataset(path +'/LS5*.nc',
                                  data_vars='minimal', coords='minimal',
                                  combine='by_coords')

#for sentinel2:
#sr_0_no_chunks = xr.open_mfdataset(path +'/s2_l2a_10m_scl_s3_39_-30*.nc',
#                                  data_vars='minimal', coords='minimal',
#                                  combine='by_coords')


In [4]:
sr_0_no_chunks

<xarray.Dataset>
Dimensions:   (time: 7, x: 1667, y: 1667)
Coordinates:
  * y         (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x         (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06
  * time      (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17
Data variables:
    crs       int32 -2147483647
    blue      (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    green     (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    red       (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    nir       (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    swir1     (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    swir2     (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    pixel_qa  (time, y, x) float32 dask.array<chunksize=(1, 1667, 1667), meta=np.ndarray>
    datase

In [5]:
sr_0_no_chunks.time

<xarray.DataArray 'time' (time: 7)>
array(['1995-02-06T16:46:48.000000000', '1995-03-10T16:45:31.000000000',
       '1995-11-21T16:34:45.000000000', '1996-01-24T16:39:02.000000000',
       '1996-03-12T16:42:17.000000000', '1996-11-07T16:55:32.000000000',
       '1996-11-23T16:56:17.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17
Attributes:
    standard_name:  time
    long_name:      Time, unix time-stamp
    axis:           T

In [6]:
sr_0 = sr_0_no_chunks.chunk({'time': -1})

In [7]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 7, x: 1667, y: 1667)
Coordinates:
  * y         (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x         (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06
  * time      (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17
Data variables:
    crs       int32 -2147483647
    blue      (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    green     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    red       (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    nir       (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    swir1     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    swir2     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    pixel_qa  (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    datase

In [8]:
sr_0['ndvi'] = ((sr_0.nir-sr_0.red)/(sr_0.nir+sr_0.red))

In [9]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 7, x: 1667, y: 1667)
Coordinates:
  * y         (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x         (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06
  * time      (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17
Data variables:
    crs       int32 -2147483647
    blue      (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    green     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    red       (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    nir       (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    swir1     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    swir2     (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    pixel_qa  (time, y, x) float32 dask.array<chunksize=(7, 1667, 1667), meta=np.ndarray>
    datase

In [10]:
begin="2019-01-1"

In [11]:
begin_time = datetime.strptime(begin, '%Y-%m-%d')

In [12]:
#if using sentinel change to 10D, if using landsat change to 80D
ts_1 = sr_0.ndvi.resample(time='80D', 
                          loffset=begin_time - 
                          datetime.utcfromtimestamp(sr_0['time'][0].values.astype(datetime)*1e-9)).nearest().interpolate_na('time')

In [13]:
ts_1

<xarray.DataArray 'ndvi' (time: 9, y: 1667, x: 1667)>
dask.array<transpose, shape=(9, 1667, 1667), dtype=float32, chunksize=(9, 1667, 1667), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01 2019-03-22 ... 2020-10-02
  * y        (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x        (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06

In [14]:
ts_1['time']

<xarray.DataArray 'time' (time: 9)>
array(['2019-01-01T00:00:00.000000000', '2019-03-22T00:00:00.000000000',
       '2019-06-10T00:00:00.000000000', '2019-08-29T00:00:00.000000000',
       '2019-11-17T00:00:00.000000000', '2020-02-05T00:00:00.000000000',
       '2020-04-25T00:00:00.000000000', '2020-07-14T00:00:00.000000000',
       '2020-10-02T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01 2019-03-22 ... 2020-10-02
Attributes:
    standard_name:  time
    long_name:      Time, unix time-stamp
    axis:           T

In [15]:
times=np.datetime_as_string(ts_1['time'],unit='D')

In [16]:
times

array(['2019-01-01', '2019-03-22', '2019-06-10', '2019-08-29',
       '2019-11-17', '2020-02-05', '2020-04-25', '2020-07-14',
       '2020-10-02'], dtype='<U28')

In [17]:
ts_1_times_clean = ts_1.assign_coords({'time':times})

In [18]:
ts_1_times_clean

<xarray.DataArray 'ndvi' (time: 9, y: 1667, x: 1667)>
dask.array<transpose, shape=(9, 1667, 1667), dtype=float32, chunksize=(9, 1667, 1667), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) <U28 '2019-01-01' '2019-03-22' ... '2020-07-14' '2020-10-02'
  * y        (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x        (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06

In [19]:
sr_0.coords

Coordinates:
  * y        (y) float64 1.527e+06 1.526e+06 1.526e+06 ... 1.477e+06 1.477e+06
  * x        (x) float64 2.027e+06 2.027e+06 2.027e+06 ... 2.077e+06 2.077e+06
  * time     (time) datetime64[ns] 1995-02-06T16:46:48 ... 1996-11-23T16:56:17

In [20]:
sr_0.crs

<xarray.DataArray 'crs' ()>
array(-2147483647, dtype=int32)
Attributes:
    grid_mapping_name:              lambert_conformal_conic
    standard_parallel:              [17.5 29.5]
    latitude_of_projection_origin:  12.0
    longitude_of_central_meridian:  -102.0
    false_easting:                  2500000.0
    false_northing:                 0.0
    semi_major_axis:                6378137.0
    semi_minor_axis:                6378136.027241431
    long_name:                      unnamed
    inverse_flattening:             6556752.3141
    crs_wkt:                        PROJCS["unnamed",GEOGCS["WGS 84",DATUM["u...
    spatial_ref:                    PROJCS["unnamed",GEOGCS["WGS 84",DATUM["u...
    GeoTransform:                   [ 2.02737e+06  3.00000e+01  0.00000e+00  ...

In [21]:
sr_0.crs.attrs['crs_wkt']

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'

In [22]:
sr_0.crs.attrs['spatial_ref']

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'

In [23]:
sr_0.crs.attrs['GeoTransform']

array([ 2.02737e+06,  3.00000e+01,  0.00000e+00,  1.52654e+06,
        0.00000e+00, -3.00000e+01])

# Computing FFT via dask 

In [24]:
import dask.array as da

In [25]:
ts_da = da.from_array(ts_1_times_clean, chunks=(9, 1667, 1667))

In [26]:
ts_da

dask.array<array, shape=(9, 1667, 1667), dtype=float32, chunksize=(9, 1667, 1667), chunktype=xarray.DataArray>

In [27]:
fft_ts_1 = da.fft.fft(ts_da,axis=0)

In [28]:
fft_ts_1

dask.array<fft, shape=(9, 1667, 1667), dtype=complex128, chunksize=(9, 1667, 1667), chunktype=numpy.ndarray>

See: 

https://github.com/CONABIO/antares3-docker/tree/master/antares3-datacube/conabio_deployment#deploy-services-of-scheduler-and-workers-with-dask--distributed-and-jupyerlab-in-scheduler

to deploy dask cluster

In [29]:
from dask.distributed import Client

In [30]:
scheduler_file = '/shared_volume/scheduler.json'

In [31]:
client = Client(scheduler_file=scheduler_file)

In [32]:
client

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 2 Cores: 2 Memory: 12.00 GB


In [33]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(fft_ts_1[:,0,0].compute())

In [ ]:
plt.plot(fft_ts_1[:,1000,1000].compute())

In [ ]:
fft_ts_datarray = fft_ts_1.compute()

In [ ]:
fft_ts_datarray

In [ ]:
from numpy import newaxis

In [ ]:
fft_ts_datarray[0,:,:].real[:,:,newaxis].shape

see: https://github.com/CONABIO/antares3/blob/rapideye-support/madmex/segmentation/__init__.py#L36 **for dimension order**

In [ ]:
fft_ts_xr = xr.DataArray(fft_ts_datarray[0,:,:].real[:,:,newaxis], dims=['x', 'y','time'],
                         coords= {'y': sr_0.coords['y'],
                                  'x': sr_0.coords['x']})



In [ ]:
fft_ts_xr.name = 'fft_numpy'

In [ ]:
fft_ts_xr = fft_ts_xr.to_dataset()

In [ ]:
fft_ts_xr

In [ ]:
from affine import Affine
from datacube.utils.geometry import CRS

See https://rasterio.readthedocs.io/en/stable/topics/migrating-to-v1.html **for relation between affine and geotransform**

In [ ]:
l_geotransform = list(sr_0.crs.attrs['GeoTransform'])

In [ ]:
l_affine = [l_geotransform[1], l_geotransform[2], l_geotransform[0],l_geotransform[4],l_geotransform[5],l_geotransform[3]]

In [ ]:
l_affine

In [ ]:
fft_ts_xr.attrs['affine'] = Affine(*l_affine)

In [ ]:
fft_ts_xr.affine

In [ ]:
fft_ts_xr.attrs['crs'] = CRS(sr_0.crs.attrs['crs_wkt'])

In [ ]:
fft_ts_xr

# Run Segmentation

In [ ]:
from django.contrib.gis.geos.geometry import GEOSGeometry
import gc
from importlib import import_module
import os

from madmex.settings import TEMP_DIR
from madmex.util import parser_extra_args


In [ ]:
algorithm = 'bis' #'slic'
datasource = 'landsat5'
name = 'fft_test_20_11_2019'
extra_args = parser_extra_args(['t=40','s=0.5','c=0.7']) #for slic: parser_extra_args(['n_segments=10000', 'compactness=0.5'])

In [ ]:
band_list = ['fft_numpy']

In [ ]:
try:
    module = import_module('madmex.segmentation.%s' % algorithm)
    Segmentation = module.Segmentation
except ImportError as e:
    raise ValueError('Invalid model argument')

In [ ]:
path = os.path.join(TEMP_DIR)

In [ ]:
path

In [ ]:
path = '/shared_volume/temp/segmentation_results/'

In [ ]:
path

In [ ]:
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
seg = Segmentation.from_geoarray(fft_ts_xr, **extra_args)

In [ ]:
seg.geobox.extent.wkt

In [ ]:
seg.segment()


In [ ]:
seg.segments_array

In [ ]:
fc = seg.polygonize()

In [ ]:
name_file = name + '.shp'

In [ ]:
shp_path = seg.to_shapefile(filename=path + '/' +name_file, fc=fc)


In [ ]:
shp_path